In [26]:
import nltk
import keras
import string
import pandas as pd
import numpy as np
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import gensim.models.keyedvectors as word2vec1
from scipy import spatial
from gensim.models import FastText

In [27]:
path = "\data\Metamorphosis_clean.txt"
text = open(path,encoding='utf-8').read().lower()
text = text.replace('\r', '').replace('\ufeff', '')
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
text = text.translate(translator)
corpus = text.split('\n')
print(corpus)
print(len(corpus))

['one morning  when gregor samsa woke from troubled dreams  he found', 'himself transformed in his bed into a horrible vermin   he lay on', 'his armour like back  and if he lifted his head a little he could', 'see his brown belly  slightly domed and divided by arches into stiff', 'sections   the bedding was hardly able to cover it and seemed ready', 'to slide off any moment   his many legs  pitifully thin compared', 'with the size of the rest of him  waved about helplessly as he', 'looked ', '', ' what s happened to me   he thought   it wasn t a dream   his room ', 'a proper human room although a little too small  lay peacefully', 'between its four familiar walls   a collection of textile samples', 'lay spread out on the table   samsa was a travelling salesman   and', 'above it there hung a picture that he had recently cut out of an', 'illustrated magazine and housed in a nice  gilded frame   it showed', 'a lady fitted out with a fur hat and fur boa who sat upright ', 'raising a heavy 

In [28]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
total_unique_words = len(tokenizer.word_index) + 1

2996
{'the': 1, 'to': 2, 'and': 3, 'he': 4, 'his': 5, 'of': 6, 'was': 7, 'in': 8, 'it': 9, 'that': 10, 'had': 11, 'a': 12, 'gregor': 13, 'as': 14, 'with': 15, 's': 16, 'she': 17, 'not': 18, 'for': 19, 'him': 20, 'her': 21, 'would': 22, 'at': 23, 'but': 24, 'on': 25, 'they': 26, 'all': 27, 'this': 28, 'be': 29, 'from': 30, 'room': 31, 'you': 32, 'if': 33, 'could': 34, 'or': 35, 'out': 36, 'have': 37, 'there': 38, 'father': 39, 'been': 40, 'sister': 41, 'so': 42, 'i': 43, 'now': 44, 'mother': 45, 'himself': 46, 'door': 47, 'gutenberg': 48, 'any': 49, 'then': 50, 'up': 51, 'back': 52, 'even': 53, 'no': 54, 'project': 55, 'into': 56, 'what': 57, 'by': 58, 'one': 59, 'work': 60, 'did': 61, 'more': 62, 'about': 63, 'their': 64, 'when': 65, 'were': 66, 'them': 67, 'way': 68, 'only': 69, 't': 70, 'do': 71, 'time': 72, 'other': 73, 'than': 74, 'tm': 75, 'is': 76, 'just': 77, 'said': 78, 'little': 79, 'without': 80, '1': 81, 'get': 82, 'still': 83, 'first': 84, 'we': 85, 'see': 86, 'some': 87, '

In [29]:
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)): 
    n_gram_seqs = token_list[:i+1]
    input_sequences.append(n_gram_seqs)

23320
[[59, 158], [59, 158, 65], [59, 158, 65, 13], [59, 158, 65, 13, 106], [59, 158, 65, 13, 106, 1079], [59, 158, 65, 13, 106, 1079, 30], [59, 158, 65, 13, 106, 1079, 30, 1537], [59, 158, 65, 13, 106, 1079, 30, 1537, 1080], [59, 158, 65, 13, 106, 1079, 30, 1537, 1080, 4], [59, 158, 65, 13, 106, 1079, 30, 1537, 1080, 4, 226], [46, 1538], [46, 1538, 8], [46, 1538, 8, 5], [46, 1538, 8, 5, 124], [46, 1538, 8, 5, 124, 56], [46, 1538, 8, 5, 124, 56, 12], [46, 1538, 8, 5, 124, 56, 12, 828], [46, 1538, 8, 5, 124, 56, 12, 828, 1539], [46, 1538, 8, 5, 124, 56, 12, 828, 1539, 4], [46, 1538, 8, 5, 124, 56, 12, 828, 1539, 4, 179], [46, 1538, 8, 5, 124, 56, 12, 828, 1539, 4, 179, 25], [5, 1540], [5, 1540, 96], [5, 1540, 96, 52], [5, 1540, 96, 52, 3], [5, 1540, 96, 52, 3, 33], [5, 1540, 96, 52, 3, 33, 4], [5, 1540, 96, 52, 3, 33, 4, 508], [5, 1540, 96, 52, 3, 33, 4, 508, 5], [5, 1540, 96, 52, 3, 33, 4, 508, 5, 97], [5, 1540, 96, 52, 3, 33, 4, 508, 5, 97, 12], [5, 1540, 96, 52, 3, 33, 4, 508, 5, 97,

In [30]:
max_seq_length = max([len(x) for x in input_sequences])
input_seqs = np.array(pad_sequences(input_sequences, maxlen=max_seq_length, padding='pre'))

18
[[   0    0    0 ...    0   59  158]
 [   0    0    0 ...   59  158   65]
 [   0    0    0 ...  158   65   13]
 ...
 [   0    0    0 ...   49  556  901]
 [   0    0    0 ...  556  901 2995]
 [   0    0    0 ...  901 2995  577]]


In [31]:
x_values, labels = input_seqs[:, :-1], input_seqs[:, -1]
y_values = tf.keras.utils.to_categorical(labels, num_classes=total_unique_words)

In [9]:
pathToBinVectors = 'cc.en.300.vec'
embed_map = word2vec1.KeyedVectors.load_word2vec_format(pathToBinVectors)

In [10]:
embed_map.get_vector('hello')

array([ 7.500e-03,  2.280e-02, -2.320e-02,  1.890e-02, -3.730e-02,
       -6.200e-03, -5.520e-02, -4.750e-02,  5.900e-02,  1.300e-03,
        7.300e-03, -1.080e-02, -1.180e-02,  6.300e-02,  4.300e-02,
        2.800e-03,  2.670e-02,  9.100e-03,  1.160e-02,  3.730e-02,
       -3.290e-02, -2.200e-03,  6.300e-03, -6.050e-02,  3.480e-02,
       -5.900e-02, -3.940e-02, -7.960e-02, -6.040e-02,  5.300e-02,
       -1.860e-02,  3.290e-02,  5.100e-03, -6.080e-02, -3.300e-02,
       -1.030e-02, -1.760e-02,  3.610e-02,  3.880e-02, -1.350e-02,
        7.400e-03, -2.740e-02,  1.890e-02, -6.840e-02, -2.820e-02,
       -5.920e-02,  2.130e-02, -3.200e-03,  5.810e-02,  2.240e-02,
       -9.000e-03,  1.730e-02,  3.300e-03,  3.840e-02, -5.000e-03,
        2.620e-02,  3.500e-02, -3.670e-02, -2.600e-02, -5.580e-02,
       -1.800e-02,  4.250e-02,  5.300e-03,  2.480e-02,  3.290e-02,
       -7.150e-02, -5.100e-03, -3.350e-02,  1.214e-01,  9.300e-03,
       -1.160e-02,  1.360e-02, -7.340e-02, -1.660e-02,  3.310e

In [11]:
embeddings_matrix = np.zeros((total_unique_words, 300))
wordss = []
for word, i in word_index.items():
  embedding_vector = embed_map.get_vector(word)
  wordss.append(word)
  if embedding_vector is not None:
    embeddings_matrix[i] = embedding_vector;

In [14]:
model = tf.keras.Sequential([
tf.keras.layers.Embedding(input_dim = total_unique_words, output_dim=300, weights=[embeddings_matrix], input_length=max_seq_length-1, trainable=False),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
tf.keras.layers.Dropout(0.2), 
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dense(total_unique_words, activation='softmax')])
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
history = model.fit(x_values, y_values, epochs=200, validation_split=0.2, verbose=1, batch_size=20)

Epoch 1/200
86/86 [==============================] - 19s 139ms/step - loss: 2.7551 - accuracy: 0.2494 - val_loss: 1.9592 - val_accuracy: 0.4755
Epoch 2/200
86/86 [==============================] - 10s 116ms/step - loss: 1.6446 - accuracy: 0.4819 - val_loss: 1.4660 - val_accuracy: 0.6037
Epoch 3/200
86/86 [==============================] - 11s 125ms/step - loss: 1.2763 - accuracy: 0.5874 - val_loss: 1.2805 - val_accuracy: 0.6247
Epoch 4/200
86/86 [==============================] - 10s 114ms/step - loss: 1.2987 - accuracy: 0.5740 - val_loss: 1.1668 - val_accuracy: 0.6387
Epoch 5/200
86/86 [==============================] - 9s 101ms/step - loss: 1.1297 - accuracy: 0.5897 - val_loss: 1.0745 - val_accuracy: 0.6340
Epoch 6/200
86/86 [==============================] - 9s 103ms/step - loss: 1.0815 - accuracy: 0.5997 - val_loss: 1.0486 - val_accuracy: 0.6853
Epoch 7/200
86/86 [==============================] - 10s 111ms/step - loss: 1.0259 - accuracy: 0.6113 - val_loss: 1.0280 - val_accuracy: 0

In [17]:
def prediction(seed_text, next_words): 
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_length-1, padding='pre')
    predicted = np.argmax(model.predict(token_list, verbose=1), axis=-1)
    ouput_word = ""

    for word, index in tokenizer.word_index.items():
      if index == predicted:  
        output_word = word
        break
      
    seed_text += ' '+output_word
    print(seed_text)

seed_phrase = "I am"
next_words = len("one two  three".split())
prediction(seed_phrase, next_words)

1/1 [==============================] - 1s 1s/step
Copier une demande
1/1 [==============================] - 0s 28ms/step
Copier une demande numéro
1/1 [==============================] - 0s 25ms/step
Copier une demande numéro numéro
